Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import csv
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

FilePath = '/content/weekly_final.csv'
# use dataset maker with numdays = 7

DF = pd.read_csv(FilePath,delimiter=',')

columns = list(DF.columns)

for i in range(len(columns)):
  if "1_" in columns[i]:columns[i] = columns[i][2:]

DF.columns = columns


DF.drop(columns=['Unnamed: 0'], inplace=True)
DF.head()

columns = DF.columns

DF = DF.dropna(subset=['RainToday'])
c = 'RainToday'

y1 = DF[['RainTomorrow']]
y2 = DF[['rainafter2days']]
y3 = DF[['rainafter3days']]
y4 = DF[['rainafter4days']]
y5 = DF[['rainafter5days']]
y6 = DF[['rainafter6days']]
y7 = DF[['rainafter7days']]
DF=DF.drop(columns = ['RainTomorrow','rainafter2days','rainafter3days','rainafter4days','rainafter5days','rainafter6days','rainafter7days'])

numerical_col = [i for i in DF.columns if (DF[i].dtype=='float64' or DF[i].dtype=='int64')]
objects_col = [i for i in DF.columns if DF[i].dtype=='object']

y=[y1,y2,y3,y4,y5,y6,y7]
for i in range(len(y)):
  y[i] = y[i].replace(to_replace={'Yes':1,'No':0})

for i in range(6):
  X_train, X_test, y_train, y_test = train_test_split(DF, y[i], test_size=0.2, stratify=y[i], random_state=0)
  t = X_train.median()
  X_train = X_train.fillna(t)
  X_test = X_test.fillna(t)

  for t_df in [X_train, X_test]:
    t_df['WindGustDir'].fillna(X_train['WindGustDir'].mode()[0], inplace=True)
    t_df['WindDir9am'].fillna(X_train['WindDir9am'].mode()[0], inplace=True)
    t_df['WindDir3pm'].fillna(X_train['WindDir3pm'].mode()[0], inplace=True)
    t_df['RainToday'].fillna(X_train['RainToday'].mode()[0], inplace=True)

  X_train = pd.concat([X_train[numerical_col], pd.get_dummies(X_train.Location,prefix='Location'),
                      pd.get_dummies(X_train.WindGustDir,prefix='WindGustDir'),
                      pd.get_dummies(X_train.WindDir9am,prefix='WindDir9am'),
                      pd.get_dummies(X_train.WindDir3pm,prefix='WindDir3pm'),
                      pd.get_dummies(X_train.RainToday,prefix='RainToday')], axis=1)

  X_test = pd.concat([X_test[numerical_col], pd.get_dummies(X_test.Location,prefix='Location'),
                      pd.get_dummies(X_test.WindGustDir,prefix='WindGustDir'),
                      pd.get_dummies(X_test.WindDir9am,prefix='WindDir9am'),
                      pd.get_dummies(X_test.WindDir3pm,prefix='WindDir3pm'),
                      pd.get_dummies(X_test.RainToday,prefix='RainToday')], axis=1)
  scaler = MinMaxScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)
  accuracies = []
  
  clf = RandomForestClassifier(n_estimators=230, random_state=0,max_depth=22)

  clf.fit(X_train, y_train)
  y_pred_test = clf.predict_proba(X_test)[:,1]
  
  thre = 0.5
  if i>0: thre = 0.38
  y_pred_test = np.where(y_pred_test>thre,1,0)

  y_pred_train=clf.predict(X_train)
  test_accu = accuracy_score(y_test, y_pred_test)
  train_acc = accuracy_score(y_train,y_pred_train)
  accuracies.append([test_accu,train_acc])
  print('Day '+str(i+1),'test accuracy score : {0:0.4f}'. format(test_accu))
  print('Model train accuracy score : {0:0.4f}'. format(train_acc))
  cm = confusion_matrix(y_test, y_pred_test)
  cm_matrix = pd.DataFrame(data=cm, columns=['Predict Negative:0', 'Predict Positive:1'], 
                                index=['Actual Negative:0', 'Actual positive:1'])
  display(cm_matrix)

v_c=y_test.value_counts()
v_c

null_accuracy = v_c[0]/sum(v_c)

print(null_accuracy)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133940 entries, 0 to 133939
Data columns (total 31 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Location        133940 non-null  object 
 1   MinTemp         133406 non-null  float64
 2   MaxTemp         133647 non-null  float64
 3   Rainfall        133100 non-null  float64
 4   Evaporation     78052 non-null   float64
 5   Sunshine        71038 non-null   float64
 6   WindGustDir     125168 non-null  object 
 7   WindGustSpeed   125208 non-null  float64
 8   WindDir9am      124700 non-null  object 
 9   WindDir3pm      130576 non-null  object 
 10  WindSpeed9am    132838 non-null  float64
 11  WindSpeed3pm    131651 non-null  float64
 12  Humidity9am     132412 non-null  float64
 13  Humidity3pm     130719 non-null  float64
 14  Pressure9am     120994 non-null  float64
 15  Pressure3pm     121023 non-null  float64
 16  Cloud9am        84643 non-null   float64
 17  Cloud3pm  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:102: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Day 1 test accuracy score : 0.8606


,Predict Negative:0,Predict Positive:1
Actual Negative:0,19980,771
Actual positive:1,2940,2929


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:102: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


...

,Predict Negative:0,Predict Positive:1
Actual Negative:0,20065,694
Actual positive:1,5115,746


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:102: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Day 6 test accuracy score : 0.7824


,Predict Negative:0,Predict Positive:1
Actual Negative:0,20074,672
Actual positive:1,5121,753


rainafter6days
0                 0.779339
dtype: float64
